In [1]:
from AnthropicWrapper import ClaudeChat, ClaudeChatCV, ClaudeChatHistory
from HumeWrapper import HumeSentimentAnalyzer
import conversationVerifier
from dotenv import load_dotenv
from joblib import load
import os

load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
# -- Global Variables --
timestamp = 1723670854
chatlog = load(f'interviews/{timestamp}/joblib/conversation.joblib') # Loading in the list of dicts

In [3]:
def reformat_chatlog(chatlog):
    dropped_context = chatlog[3:]
    outputchatlog = []

    for i in range(0, len(dropped_context), 2):
        if i + 1 < len(dropped_context):
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': dropped_context[i+1]['content']
            }

            outputchatlog.append(tempdict)
        else:
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': 'System - The candidate Ended the Interview'  
            }
            outputchatlog.append(tempdict)
            break 

    return outputchatlog

chatlog_chat = reformat_chatlog(chatlog)
# for item in chatlog_chat:
#     print(item)

In [4]:
# Initialising the Sentiment Analyser
sentiment_analyser = HumeSentimentAnalyzer(api_key=os.getenv("HUME_API_KEY"))

In [5]:
# Initialising Sentiment Summariser
chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
Model your answer beginning with something along the lines of "For this particular response, the candidate...".
"""

sentiment_summariser = ClaudeChat(chat_model_name, system_prompt)

In [6]:
# Initialising Evaluation LLM
chat_model_name = "claude-3-5-sonnet-20240620"
system_prompt = "You are a highly skilled ."

candidate_evaluator = ClaudeChatHistory(chat_model_name, system_prompt)

In [7]:
conversationVerifier.process_qa_pair(chatlog_chat)

Breakpoint 1
Breakpoint 2
Breakpoint 3
Breakpoint 4


[{'interviewer': "Hello! Thank you for taking the time to speak with me today about the Entry-Level RAG AI Engineer role. I'm looking forward to learning more about your experience and skills. To start off, could you tell me about any experience you have with setting up and optimizing retrieval-augmented generation (RAG) pipelines?",
  'candidate': " I'm sure. So I'm currently working on my final year dissertation which is to build a LLM screening system that will screen candidates and assess whether they are fit to move on to the next round. So to achieve this one of the steps involved checking the accuracy of the candidates answer. To check the accuracy of the answer what I basically did was set up a rag pipeline. That pipeline involved getting each, getting the transcript of each question and answer and then dividing the answer into searchable queries using an LLM, using those searchable queries to web script Google links and put it in the context window of a vector store. Now I cre

## Now Analysing the post-conversation to make a decision

In [8]:
# Specify the filepath
filepath = f'interviews/{timestamp}/audio'

# Get all files in the filepath
files = [f for f in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, f))]

# Loop through each file
sentiments = []
for count, file in enumerate(files, 1):
    # Print the file name
    print(os.path.join(filepath, str(file)))
    result = sentiment_analyser.analyze_audio(os.path.join(filepath, str(file)))
    sentiment_summary = sentiment_summariser.chat(str(result))
    sentiments.append((result, sentiment_summary))
    chatlog_chat[count-1]['sentiment'] = sentiment_summary

interviews/1723670854/audio\audio_1_1723670854.wav
Analyzing audio...
interviews/1723670854/audio\audio_2_1723670854.wav
Analyzing audio...
interviews/1723670854/audio\audio_3_1723670854.wav
Analyzing audio...
interviews/1723670854/audio\audio_4_1723670854.wav
Analyzing audio...
interviews/1723670854/audio\audio_5_1723670854.wav
Analyzing audio...


In [10]:
chatlog_chat

[{'interviewer': "Hello! Thank you for taking the time to speak with me today about the Entry-Level RAG AI Engineer role. I'm looking forward to learning more about your experience and skills. To start off, could you tell me about any experience you have with setting up and optimizing retrieval-augmented generation (RAG) pipelines?",
  'candidate': " I'm sure. So I'm currently working on my final year dissertation which is to build a LLM screening system that will screen candidates and assess whether they are fit to move on to the next round. So to achieve this one of the steps involved checking the accuracy of the candidates answer. To check the accuracy of the answer what I basically did was set up a rag pipeline. That pipeline involved getting each, getting the transcript of each question and answer and then dividing the answer into searchable queries using an LLM, using those searchable queries to web script Google links and put it in the context window of a vector store. Now I cre